# `Mict` -- middle-ground between `dict` and a `class`

Provides `MATLAB`-like setting/storage of items in a dict (dictionary) and a handful of interactivity tools.

`mict` is intended to be a middle ground between `dict` and full fledged `class` / `object` pattern for structured data storage.

It does a bit more than basic `dict`, but does not attempt to superseede pandas, numpy nor other advanced data storage tools



In [ ]:
## Installation

try installing with `pip`, grabbing it straight from git:
    
```bash
python -m pip install git+https://github.com/<username>/<repo_name>.git
```

## Basic usage

In [1]:
from mict import mict
q=mict()
q.first = 'Hello world!'
q

key,value
first,Hello world!


---

the key power of mict is the ease of adding, removing, altering the contents -- and the the nice visualisation of them.

You can add new keys easily, and then access them using dot-notation:

In [2]:
q.second = 'not'
q.third = 3
q

key,value
first,Hello world!
second,not
third,3


--- 

The visualiser function in `mict` is called `reprstyler`.

A new `mict` instance comes with the `reprstyler_html` already set to `reprstyler_basic_html` but you can, and should make your own reprstylers.

example reprstyler:

## Convinence save and load functions using pickle

methods `to_pickle` and `from_pickle` work as you would expect:

In [3]:
q.to_pickle('demo.pickle')

True

In [4]:
q=mict.from_pickle('demo.pickle')
q

key,value
first,Hello world!
second,not
third,3


## Customizing the reprstyler 

This is the real reason why I developed mict: I wanted to have a quick and simple way to customize how the contents of the dictionary are visualized.
    

In [12]:
from mict import mict
def baldstyler(subject):
    return f'baldness score: {subject.bald:0.3f}'

q=mict(zap=None,bald=6.28,hidden_value1=45,reprstyler_html=None,reprstyler=baldstyler)

q

baldness score: 6.280

---

As everything in `mict`, the reprstyler can be changed **after** initializing the `mict` storage itself

In [13]:
def new_reprstyler(subject=None):
    txt = 'keys:  '
    for key in subject.keys():
        if key == 'reprstyler' or key == 'reprstyler_html':  # do not list reprstyler
            continue
        txt = f"{txt} {key} ={subject[key]};"
    return txt

q.reprstyler = new_reprstyler
q

keys:   zap =None; bald =6.28; hidden_value1 =45;

then, if you want a different styler for jupyter, and a different one for text-only, you can (optionally) override the `reprstyler` with `reprstyler_html`. It will only be used if `_repr_html_` is called -- as `jupyter` first tries to call that first


In [14]:
def baldstyler_html(subject):
    return f'<h4>✅baldness score: {subject.bald:0.3f}</h4>' if subject.bald>5 else f'<h4>❌not bald enough: {subject.bald:0.3f}</h4>'
q.reprstyler_html = baldstyler_html
q

keys:   zap =None; bald =6.28; hidden_value1 =45;

In [15]:
q.bald = 3
q

keys:   zap =None; bald =3; hidden_value1 =45;

## An even quicker way to make a reprstyler

You can use `lambda` anonymous function definition to make a reprstyler quickly.

Note that reprstyler_html will display any html you give it, including images, videos, sound or javascript inserts

In [16]:
q=mict(fill='blue',r=15)
q.reprstyler_html= lambda this:f'<svg height="100" width="100"><circle cx="50" cy="50" r="{this.r}" stroke="black" stroke-width="3" fill="{this.fill}" /></svg>'
q


{'fill': 'blue', 'r': 15, 'reprstyler_html': <function <lambda> at 0x0000020D0218D040>}

In [17]:
q.r=32
q.fill='red'
q

{'fill': 'red', 'r': 32, 'reprstyler_html': <function <lambda> at 0x0000020D0218D040>}

In [27]:
## Remove the `reprstyler` and use the default `dict` styler.
q.reprstyler_html=None
try:
    q.pop('reprstyler_html')
except:
    pass

In [28]:
q

{'fill': 'red', 'r': 32}

---

## Note

If you really feel like it, this could be extended to markdown, png, svg and other visualizers supported by jupyter.


Importantly, you can still see the classic `dict` __repr__ function (lists all keys/values) using

```python
super(handybeam.dict,q).__repr__()
```

# Advanced uses

## Show shape of big variables, instead of it's content.

many of the actual research code will use numpy arrays or long lists. These are typically unwieldy to just display.

`mict` provides a way to only display the shape of the numpy/tensorflow/pytorch/jax array, instead of the contents. You will find that this is often what you need displayed.

In [ ]:
from mict import reprstyler_basic_html
import numpy
q=mict(small_array=numpy.array([1,2,3,4,5]))
q

In [ ]:
q.large_array=numpy.random.random((150,150))
q.reprstyler_html = reprstyler_basic_html # optional set -- this is already a default
q

# Advanced uses

## Capture locals from inside the function and return them in a `dict` / `mict` 

When developing a research function, you will often want to capture **all** the locals inside it for debug purposes. 

Only when stabilizing the implementation, you will want to prune the result and leave usefull return values only.

In [ ]:
 # define a function
def do_maths(x=1,y=2):
    a=x+y
    b=a*x
    c=b*y
    result = mict.from_locals()  # puts `x`,`y`,`a`,`b`,`c` into `result`. 
    result.pop('b') # remove 'b' from the result
    return result

# execute that function
demo_result = do_maths(x=4,y=2)

# review function's locals
demo_result

# Advanced uses

## Nested dictionary use and special attributes

note that `name` and `type` have a special meaning for the default visualizer, `reprstyler_basic_html`.

Moreover, the `reprstyler_basic_html` will try to obtain html from inner (nested) `mict`s and display it in a table.

Note that `reprstyler_basic_html` is merely another function from the `mict` module, and can be overridden with any other reprstyler. If `reprstyler_html` is set to `None`, the default `dict.__repr__()` is used.

In [37]:
x=mict(name='first',type='coordinate', value=3)
y=mict(name='second', type='coordinate', value=5)
point=mict(type='outer <b>mict</b>',x=x,y=y)

In [38]:
x

key,value
value,3


In [39]:
y

key,value
value,5


In [40]:
point

{'type': 'outer <b>mict</b>', 'x': {'name': 'first', 'type': 'coordinate', 'value': 3, 'reprstyler_html': <function reprstyler_basic_html at 0x0000020D02167310>}, 'y': {'name': 'second', 'type': 'coordinate', 'value': 5, 'reprstyler_html': <function reprstyler_basic_html at 0x0000020D02167310>}, 'reprstyler_html': <function reprstyler_basic_html at 0x0000020D02167310>}

In [41]:
v=point._repr_html_()
v

'<em>Type:</em> outer <b>mict</b>; <br/><table><tr><th>key</th><th>value</th></tr><tr><td>x</td><td><em>Type:</em> coordinate; <em>Name:</em> first; <br/><table><tr><th>key</th><th>value</th></tr><tr><td>value</td> <td> 3</td> </tr>  </table></td> </tr> <tr><td>y</td><td><em>Type:</em> coordinate; <em>Name:</em> second; <br/><table><tr><th>key</th><th>value</th></tr><tr><td>value</td> <td> 5</td> </tr>  </table></td> </tr>  </table>'

In [42]:
point.reprstyler_html=None

In [43]:
point

{'type': 'outer <b>mict</b>', 'x': {'name': 'first', 'type': 'coordinate', 'value': 3, 'reprstyler_html': <function reprstyler_basic_html at 0x0000020D02167310>}, 'y': {'name': 'second', 'type': 'coordinate', 'value': 5, 'reprstyler_html': <function reprstyler_basic_html at 0x0000020D02167310>}, 'reprstyler_html': None}

In [44]:
isinstance(point,mict)

True

## More examples

In [45]:
from mict import mict
import numpy
import math
q=mict(title="some title",subtitle="some subtitle",interesting_integer = 3,interesting_float = math.tau,  big_array=numpy.random.random((200,250)))
q

key,value
title,some title
subtitle,some subtitle
interesting_integer,3
interesting_float,6.283185307179586
big_array,"np.array(shape=(200, 250))"


In [46]:
def basic_html_styler(this):
    output = f'<h1>{this.title}</h1>'
    output = f'{output}<h2>{this.subtitle}</h2>'
    output = f'{output}<p>interesting integer:{this.interesting_integer:04d}</p>'
    output = f'{output}<p>interesting float:{this.interesting_float:0.{this.interesting_integer}f}</p>'
    output = f'{output}<hr/>'
    return output

q.reprstyler_html = basic_html_styler
q

{'title': 'some title', 'subtitle': 'some subtitle', 'interesting_integer': 3, 'interesting_float': 6.283185307179586, 'big_array': array([[0.40459763, 0.64140439, 0.02250562, ..., 0.33400559, 0.17766872,
        0.03943357],
       [0.43986926, 0.27937355, 0.46742819, ..., 0.37697844, 0.21098662,
        0.09522587],
       [0.36339104, 0.01929237, 0.18907834, ..., 0.5696711 , 0.59096884,
        0.34467516],
       ...,
       [0.23138886, 0.87595231, 0.97926423, ..., 0.17845339, 0.96092345,
        0.2536023 ],
       [0.43489639, 0.58179511, 0.17412641, ..., 0.98394694, 0.14202521,
        0.28503678],
       [0.94831272, 0.67236216, 0.64096975, ..., 0.49160768, 0.21575886,
        0.0684129 ]]), 'reprstyler_html': <function basic_html_styler at 0x0000020D0218D790>}

## Gotchas

* `mict` does not throw an error when trying to access undefined field. Instead, it returns `None`


## Attributions

Happily copypasted from https://stackoverflow.com/questions/2352181/how-to-use-a-dot-to-access-members-of-dictionary , and modified only slightly.

then, extended a bit.

then, a bit more, with optional reprstyler. See the source code for `self.__repr__()`.


## Related packages 

see also: 

https://pypi.org/project/python-box/


## License

MIT License, Copyright (c) 2017-2020 Chris Griffith. See LICENSE file.
